In [1]:
import gym
import os
import torch
import argparse
import pickle
import numpy as np
from tqdm import tqdm
import sys
sys.path.append("./dssm")
from train import main as train_ssm
from diayn.examples.diayn import experiment
from rlkit.envs.wrappers import NormalizedBoxEnv
from rlkit.samplers.util import DIAYNRollout as rollout


def collect(sim, policy, depth, args):
    data = []
    for skill in range(policy.stochastic_policy.skill_dim):
        for trial in tqdm(range(100)):
            # print("skill-{} rollout-{}".format(skill, trial))
            path = rollout(
                sim,
                policy,
                skill,
                max_path_length=args.H,
                render=False,
            )
            data.append([path['actions'], path['next_observations']])

    train_data = data[:int(len(data)*0.9)]
    test_data = data[int(len(data)*0.9):]

    train_path = os.path.join(args.data_dir, "./train{}.pkl".format(depth))
    test_path = os.path.join(args.data_dir, "./test{}.pkl".format(depth))
    os.makedirs(args.data_dir, exist_ok=True)
    with open(train_path, mode='wb') as f:
        pickle.dump(train_data, f)
    with open(test_path, mode='wb') as f:
        pickle.dump(test_data, f)


def update_sim(sim, depth):
    ssm = train_ssm("--H {} --depth {}".format(args.H, depth))
    # return Sim(sim)


def update_policy():
    pass



def Sim(env):
    def step(action):
        lb = env._wrapped_env.action_space.low
        ub = env._wrapped_env.action_space.high
        scaled_action = lb + (action + 1.) * 0.5 * (ub - lb)
        scaled_action = np.clip(scaled_action, lb, ub)

        wrapped_step = env._wrapped_env.step(scaled_action)
        next_obs, reward, done, info = wrapped_step
        if env._should_normalize:
            next_obs = env._apply_normalize_obs(next_obs)
        return next_obs, reward * env._reward_scale, done, info

    env.step = step
    return env


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('env', type=str,
                        help='environment')
    parser.add_argument('file', type=str,
                        help='path to the snapshot file')
    parser.add_argument("--data_dir", type=str,
                        default="./data/")
    parser.add_argument('--skill_dim', type=int, default=10,
                        help='skill dimension')
    parser.add_argument('--H', type=int, default=300,
                        help='Max length of rollout')
    parser.add_argument('--D', type=int, default=1,
                        help='Depth (The number of update)')
    args = parser.parse_args("HalfCheetah-v2 diayn/data/DIAYN-10-HalfCheetah-v2/DIAYN_10_HalfCheetah-v2_2021_02_23_11_50_33_0000--s-0/params.pkl --H 100".split())


    data = torch.load(args.file)
    policy = data['evaluation/policy']
    env = NormalizedBoxEnv(gym.make(str(args.env)))
    sim = Sim(NormalizedBoxEnv(gym.make(str(args.env))))

    for depth in range(args.D):
        collect(sim, policy, depth, args)
        sim = update_sim(sim, depth)
        #print(data[0].keys(), len(data))
        # sim = update_sim(data)
        # policy = update_policy(sim, policy

No personal conf_private.py found.
doodad not detected


[I 210223 20:27:09 train:47] args: Namespace(B=64, H=100, T=10, a_dim=6, data_dir='./data/', depth=0, device=[0], epochs=20, h_dim=128, iters_to_accumulate=1, load_epoch=None, o_dim=17, s_dim=64, seed=0, timestamp='Feb23_20-27-09')


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(900, 100, 6) (900, 100, 17)
(100, 100, 6) (100, 100, 17)


[I 210223 20:27:12 trainer:53] (train) Epoch: 1 {'s_loss': 30850646.13392857, 'x_loss': 285.2186791556222, 's_aux_loss': 56.47733579363142, 'loss': 30850932.01785714}
[I 210223 20:27:12 trainer:53] (test) Epoch: 1 {'s_loss': 201723.515625, 'x_loss': 238.38076782226562, 's_aux_loss': 47.83351135253906, 'loss': 201961.890625}
[I 210223 20:27:13 trainer:53] (train) Epoch: 2 {'s_loss': 100746.994140625, 'x_loss': 258.872193472726, 's_aux_loss': 42.69347218104771, 'loss': 101005.86607142857}
[I 210223 20:27:13 trainer:53] (test) Epoch: 2 {'s_loss': 39650.51171875, 'x_loss': 215.9703826904297, 's_aux_loss': 36.18363571166992, 'loss': 39866.48046875}
[I 210223 20:27:14 trainer:53] (train) Epoch: 3 {'s_loss': 25596.91476004464, 'x_loss': 247.93915448869978, 's_aux_loss': 33.75431292397635, 'loss': 25844.854352678572}
[I 210223 20:27:14 trainer:53] (test) Epoch: 3 {'s_loss': 15465.57421875, 'x_loss': 207.56192016601562, 's_aux_loss': 30.281095504760742, 'loss': 15673.1357421875}
[I 210223 20:27